# MSIII Computations

We just changed several things in our MSII orbit to address issues from the second milestone and make instructor-recommended corrections. A brief summary is below, but the MSII_Base_Orbit_Determination_corrected.ipynb and MSII_corrected.ipynb files can be consulted for more specific details about the implementation.
- Used maximum latitude instead of average (see Formula_Sheet.py)
- Changed max resolution to 3.99m (see MSIII_useful_consts.py)
- Added visits-per-day metric (see MSIII_Constellation.py)
- Used visits-per-day as metric for orbit template (MSII_Base_Orbit_Determination_corrected.ipynb)
    - This changed period to 2h and altered COEs significantly
- Forced the cheapest camera for budget reasons (see MSIII_Components.py)
    - This allowed cheaper structure and possibly changed ADCS
- Recomputed constraints to ensure they are met (se MSII_corrected.ipynb)

Note that the comments in all of these files have not necessarily changed, but the functionality has. The emphasis was on quick corrections to move on to MSIII, but the changes are fairly minor and should be understandeable without updated comments.

Now, we have several orders of business before we can truly begin MSIII.

In [104]:
#  import everything (our entire library)
from MSIII_Components import *
from MSIII_useful_consts import *
from MSIII_Constellation import MSIII_Constellation, MSIII_Satellite
from MSIII_Constraints import *
from MSIII_Orbit import MSIII_Orbit
from pretty_outputs import indent_section
from Formula_Sheet import *
from CoolGeometry import *

### Several orders of business...
We need to load the MSII corrected constellation into our program. Then, we need to somewhat nerf (make less powerful) it in order to meet the constraint of having all the same COEs except for true anomaly. Be advised that this will have HORRENDOUS impacts on performance. Now, the points only cross an orbital plane at most 2 times per day. Before, they crossed one at most six times per day. Having a single orbital plane for the entire constellation is the single worst thing you could do for point coverage. 

But, we must see to the needs of the Space Force. Single orbital plane it is...

In [105]:
# Here is the summary of our starting corrected MSII constellation
"""
Found constellation with 0.04225% coverage
Constellation #1:
	Payload:   Tiny Cam
	Structure: Option I
	ADCS:      Option II
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 290.375
		w    : 90
		v    : 0.0
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 300.375
		w    : 90
		v    : 126.19563368939261
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 310.375
		w    : 90
		v    : 132.1759947994688

Constellation #1 Comprehensive Summary: 
	Constellation #1 Concise Summary:
		Passes:       True
		In Budget:    True
		Viable Orbit: True
		Volume Fits:  True
	
	Constellation #1 Orbit Viability: VIABLE
		ORBIT 1
			CATASTROPHIC?      False
			DRAGLESS?          True
			CRASHLESS?         True
			Max Altitude:      2760.7213975942896 km
			Min Altitude:      601.0 km
			Is Geosynchronous: False
		
		ORBIT 2
			CATASTROPHIC?      False
			DRAGLESS?          True
			CRASHLESS?         True
			Max Altitude:      2760.7213975942896 km
			Min Altitude:      601.0 km
			Is Geosynchronous: False
		
		ORBIT 3
			CATASTROPHIC?      False
			DRAGLESS?          True
			CRASHLESS?         True
			Max Altitude:      2760.7213975942896 km
			Min Altitude:      601.0 km
			Is Geosynchronous: False
		
	
	Constellation #1 Budgeting:
		Margin Meets Budget:  True
		Certain Cost:        $16365000
		Est. Cost:           $20540000.000000004
		Cost With Margin:    $22594000.000000007
		Budget:              $225000000
		Meets Budget:         True
		Unallocated Money:   $188180454.54545453
	
	Constellation #1 Volume Summary:
		Payload Fits:     True
		Available Volume: 1.5 m^3
		Used Volume:      0.216 m^3
		Remaining Volume: 1.284 m^3
	
	Constellation #1 Visitation Summary:
		Average visitations per day: 8.0
	
	Constellation #1 Point Coverage:
		Can Point Within SW:     True
		% Coverage:              0.04225%
		Points Sampled:          4000
		Points in range and LOS: 169
		Points in range:         169
		Points in LOS:           675
		Closest Approach:        598.0135220827832 km
"""


# load in our orbits from the corrected MSII 
old_orbit1 = MSIII_Orbit( 
	a    = 8058.997698797144,
	e    = 0.13399441706731352,
	i    = 53.25,
	raan = 290.375,
	w    = 90,
	v    = 0.0
)
old_orbit2 = MSIII_Orbit( 
	a    = 8058.997698797144,
	e    = 0.13399441706731352,
	i    = 53.25,
	raan = 300.375,
	w    = 90,
	v    = 126.19563368939261
)
old_orbit3 = MSIII_Orbit( 
	a    = 8058.997698797144,
	e    = 0.13399441706731352,
	i    = 53.25,
	raan = 310.375,
	w    = 90,
	v    = 132.1759947994688
)

old_sat1 = MSIII_Satellite() # create new satellites for our constellation
old_sat2 = MSIII_Satellite()
old_sat3 = MSIII_Satellite()
old_sat1.orbit = old_orbit1 # force the MSII corrected orbits into the new satellites
old_sat2.orbit = old_orbit2
old_sat3.orbit = old_orbit3



const_II = MSIII_Constellation({
	"payload": PAYLOADS[0],          # Payload TinyCam (cheapest)
	"structure": STRUCTURES[0],      # Structure Option I
	"adcs": ADCS[1],                 # ADCS Option II
	"sats": []                       # no satellites, we will add these in next
})
const_II.sats = [old_sat1, old_sat2, old_sat3] # force the correct satellites into the constellation

Using default satellite constructor, make sure to add orbits.
Using default satellite constructor, make sure to add orbits.
Using default satellite constructor, make sure to add orbits.


In [106]:
# now we need to nerf the orbits (put them all on the same orbit, except for true anomaly)

def nerf_constellation(c : MSIII_Constellation):
    orbit1 = c.sats[0].orbit # pick the first orbit as the dominant orbit
    orbit1 : MSIII_Orbit
    for satellite in c.sats: # force all the other orbit to the dominant orbit's COEs
        orbit = satellite.orbit
        orbit : MSIII_Orbit
        orbit.a = orbit1.a
        orbit.e = orbit1.e
        orbit.raan = orbit1.raan
        orbit.i = orbit1.i
        orbit.w = orbit1.w

const3 = const_II
const3 : MSIII_Constellation
nerf_constellation(const_II) # nerf it


# assess the damage
ner_name = "(Nerfed)"
print("="*10 + "Post-nerf damage assessment" + "="*10)
print("\n")
print(const3.to_string(ner_name))
print(const3.get_concise_summary().to_string(ner_name))
print(const3.assess_visitation().to_string(ner_name))

==========Post-nerf damage assessment==========


Constellation (Nerfed):
	Payload:   Tiny Cam
	Structure: Option I
	ADCS:      Option II
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 290.375
		w    : 90
		v    : 0.0
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 290.375
		w    : 90
		v    : 126.19563368939261
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 290.375
		w    : 90
		v    : 132.1759947994688

Constellation (Nerfed) Concise Summary:
	Passes:       True
	In Budget:    True
	Viable Orbit: True
	Volume Fits:  True

Constellation (Nerfed) Visitation Summary:
	Average visitations per day: 8.5



This is strange, our visitation numbers actually went up. Upon further review of the visitation assessment implementation, it is still completely correct. Perhaps this was not the most reliable metric, but we will be sticking with it at this point to reduce design complexity. Possible reasons why this visitation number might have gone up include the potential for a point to briefly exit and then re-enter the swath width at perigee, thus counting two visitations. This is unclear, but will not be investigated further. A better metric to avoid this issue may have been to calculate the average squared duration between visits (squared so that longer waits would be penalized more heavily) and minimize that number. 

All this discussion aside, we now have our orbits.

#### Decide and Load the Parking Orbit + Launch Choices

In [107]:
"""
We originally picked the Mid-Atlantic Regional Spaceport because Tate tested all of them in a 
MATLAB script and this is the one that worked.
"""
site = LAUNCH_SITES[2]

"""
At this point, we should also load in our parking orbit. For now, we will be using the target
inclination. This allows us to minimize the magnitude of plane changing we will need to do
later.

RAAN and w were both set to the exact value of the target orbit. This will minimize the magnitude of
maneuvers we have to do. Note that the value of w is irrelevant, because this orbit is almost 
completely circular. (e ~= 0)
"""
park = MSIII_Orbit(
    a = 7080,                          # chosen for us
    e = 0.03,                          # chosen for us
    i = const3.sats[0].orbit.i,        # equal to the target orbit i (minimize maneuvers)
    raan = const3.sats[0].orbit.raan,  # equal to the target orbit raan (minimize maneuvers)
    v = 0,                             # chosen for us
    w = const3.sats[0].orbit.w         # equal to the target orbit w to minimize maneuver effort
)

## Steps
Now it is time to actually work through the checklist.

### Step 1
See the corrections explained at the top of this document. 

### Step 2
Launch site and orbit were selected simultaneously right before this section (see "Decide and Load the Parking Orbit + Launch Choices)

In [108]:
# Output the selected parking orbit. Ignore w, this had to be included for the 
# orbit library to work.
print("PARKING " + park.to_string()) 

PARKING ORBIT: 
	a    : 7080
	e    : 0.03
	i    : 53.25
	raan : 290.375
	w    : 90
	v    : 0


### Step 3
#### a.

In [109]:
"""
Note that the launch site has already been selected in the section "Decide and Load the Parking 
Orbit + Launch Choices"
"""

# just output the chosen launch site
import json
print(json.dumps(site, indent=2))

{
  "name": "Mid-Atlantic Regional Spaceport",
  "latitude": 37.8,
  "longitude": 75.5,
  "min_azimuth": 90,
  "max_azimuth": 160
}


#### b.

In [110]:
launch_azimuths = {}

# from formula sheet, we are launching directly in
alpha = park.i 
assert(alpha >= site["latitude"]) # otherwise, there are no opportunities

if (site["latitude"] < alpha):
    # yay! there are two opportunities
    
    # from formulasheet.py
    # correct_angle just shifts the angle around by 360 degrees until it is between 0 and 360
    launch_azimuths["AN"] = correct_angle(compute_gamma(alpha, site["latitude"])) 
    launch_azimuths["DN"] = correct_angle(180 - launch_azimuths["AN"])
else:
    # darn! only one opportunity
    launch_azimuths["the only"] = correct_angle(compute_gamma(alpha, site["latitude"]))

for azimuth_type in launch_azimuths.keys():
    print(f"Found {azimuth_type} azimuth: {launch_azimuths[azimuth_type]} degrees")


Found AN azimuth: 40.779891701282125 degrees
Found DN azimuth: 139.2201082987179 degrees


#### c. 

In [111]:
LWSTs = {}

if (site["latitude"] < alpha):
    # yay! there are two opportunities
    gamma = compute_gamma(alpha, site["latitude"])
    delta = compute_lowercase_delta(gamma, alpha)
    LWSTs["AN"] = correct_angle(park.raan + delta)
    LWSTs["DN"] = correct_angle(park.raan + 180 - delta)
else:
    # darn! only one opportunity
    gamma = compute_gamma(alpha, site["latitude"])
    delta = compute_lowercase_delta(gamma, alpha)
    LWSTs["the only"] = correct_angle(park.raan)

for lwsts_type in LWSTs.keys():
    print(f"Found {lwsts_type} LWST: {LWSTs[lwsts_type]} degrees")


Found AN LWST: 309.45741954192135 degrees
Found DN LWST: 91.29258045807865 degrees


#### d. 

In [112]:
lst_ready_by = 9 * (360 / 24)
print(f"Ready by {lst_ready_by} degrees LST")

launch_waits = {}
for lwsts_type in LWSTs.keys():
    wait = correct_angle(LWSTs[lwsts_type] - lst_ready_by)
    launch_waits[lwsts_type] = wait
    print(f"Found {lwsts_type} wait time: {wait} degrees ST")

Ready by 135.0 degrees LST
Found AN wait time: 174.45741954192135 degrees ST
Found DN wait time: 316.29258045807865 degrees ST


#### e.

In [113]:
# put it all together
opportunities = []

for launch_type in LWSTs.keys():
    time = LWSTs[launch_type]
    wait = launch_waits[launch_type]
    azim = launch_azimuths[launch_type]
    allowed = (azim < site["max_azimuth"]) and (azim > site["min_azimuth"])
    opportunities.append({
        "Launch": launch_type,
        "LWST": time,
        "wait": wait,
        "azimuth": azim,
        "allowed": allowed
    })
    
print(json.dumps(opportunities, indent=2))

[
  {
    "Launch": "AN",
    "LWST": 309.45741954192135,
    "wait": 174.45741954192135,
    "azimuth": 40.779891701282125,
    "allowed": false
  },
  {
    "Launch": "DN",
    "LWST": 91.29258045807865,
    "wait": 316.29258045807865,
    "azimuth": 139.2201082987179,
    "allowed": true
  }
]


Suffice it to say, we will launch into the Descending Node because that is the only one for which our azimuth is permitted by the launch site.

In [114]:
launch = opportunities[1] # we just picked the second opportunity

#### f.

In [115]:
v_ls = 0.4651 * math.cos(math.degrees(site["latitude"])) # from formula sheet
r_ls = R_EARTH
r_bo = park.a * (1 - park.e) # from the formula sheet
v_bo = math.sqrt(2 * ((MIU / r_bo) - (MIU / (2 * park.a)))) # from the formula sheet

v_pe_gain = math.sqrt((2 * MIU * (r_bo - r_ls)) / (r_bo * r_ls)) # from formula sheet
v_losses_otg = 1.1 # from checklist
phi = 0 # in radians

v_needed = [ # from the formula sheet
    -v_bo * math.cos(phi) * math.cos(math.radians(launch["azimuth"])),
    v_bo * math.cos(phi) * math.sin(math.radians(launch["azimuth"])) - v_ls,
    v_bo * math.sin(phi) + v_pe_gain
]

v_design = mag_of(v_needed) + v_losses_otg # from the formula sheet

print(f"Computed Initial Launch Delta V required for design: {v_design} km s^-1")

Computed Initial Launch Delta V required for design: 9.485448670259506 km s^-1


### Step 4

In [116]:
"""
Because of the way we implemented MSII, this problem is trivial. See the MSIII_Orbit.py file for 
implementation details on this function.
"""

v_48hours = park.get_v_at_time(h_to_sec(48))
print(f"True Anomaly at t=48h : {v_48hours} degrees")

True Anomaly at t=48h : 53.93385140404728 degrees


#### Step 5

#### a. 

In [117]:
park.v = v_48hours # move the parking orbit model to reflect t=48h
park.recomp_meta_vars() # allow the orbit to recompute cached values
time_to_perigee = park.get_t_until_v(0) # see MSIII_Orbits.py for implementation
# note that this implementation is almost verbatum from the formula sheet

total_time = h_to_sec(48) + time_to_perigee
print(f"We need to wait a total time of {total_time} s until we can begin maneuvers")
print(f"That is {sec_to_h(total_time)} hours")

park.v = 0 # update the position to reflect that we are at perigee
park.recomp_meta_vars()

We need to wait a total time of 177885.66071574867 s until we can begin maneuvers
That is 49.41268353215241 hours


#### b.

In [118]:
transfer = park.duplicate() # copy the parking orbit as a baseline
target : MSIII_Orbit = const3.sats[0].orbit.duplicate()

target_apogee = target.a * (1 + target.e) # from formula sheet

"""
Because all of our COEs except for eccentricity and semimajor axis are identical between parking 
orbit and transfer orbit, we only need a hohmann transfer. 

To state it differently, a hohmann transfer is "the most efficient method to complete the maneuver"
"""

# rbo just happens to be the radius at perigee, which is where we are at
transfer.a = (r_bo + target_apogee) / 2
transfer.e = (target_apogee / transfer.a) - 1 # derived from formula sheet

print("TRANSFER " + transfer.to_string())

TRANSFER ORBIT: 
	a    : 8003.229198797144
	e    : 0.1418963733998555
	i    : 53.25
	raan : 290.375
	w    : 90
	v    : 0


#### c. 

In [119]:
v_transfer = transfer.get_vel()
v_park = park.get_vel()
standard_delta_v_1 = abs(v_transfer - v_park)

transfer.v = 180 # we wait until we get to apogee
transfer.recomp_meta_vars()
target.v = 180 # we will maneuver into apogee of the mission orbit
target.recomp_meta_vars()

v_transfer_2 = transfer.get_vel()
v_target_2 = target.get_vel()
standard_delta_v_2 = abs(v_transfer_2  - v_target_2)

total_hohmann_delta_v = standard_delta_v_1 + standard_delta_v_2

print(f"Standard delta V needed to start hohmann: {standard_delta_v_1} km s^-1")
print(f"Standard delta V needed to end hohmann:   {standard_delta_v_2} km s^-1")
print(f"Total delta V needed to perform hohmann:  {total_hohmann_delta_v} km s^-1")




Standard delta V needed to start hohmann: 0.4091590157271252 km s^-1
Standard delta V needed to end hohmann:   0.02810355936321951 km s^-1
Total delta V needed to perform hohmann:  0.4372625750903447 km s^-1


#### d. 

In [120]:
# bit of a shortcut, but we already have the functionality built in, so why not
transfer.v = 0
tof_hohman = transfer.get_t_until_v(180) # use orbit prediction to solve tof_hohmann
print(f"The transfer from perigee of park to apogee of mission is {tof_hohman} s")
print(f"That is {sec_to_h(tof_hohman)} hours.")

The transfer from perigee of park to apogee of mission is 3562.6965638580923 s
That is 0.9896379344050257 hours.


#### e.

In [121]:
# assumes both orbits are coorbital
# assumes chaser is at apogee
# accepts a maneuverable spacecraft and a target one, as well as a max number of periods we are 
# willing for this phase shift to take place over.
# yes, this is somewhat unconventional, but I wanted to try something new. This should still
# work just fine, though, and it allows us to split up the rendezvous over several periods.
def get_total_coorbital_rendezvous_delta_v(chaser : MSIII_Orbit, target : MSIII_Orbit, periods):
    assert(chaser.v == 180) # ensure we are in fact at apogee
    assert(chaser.i == target.i and # ensure we are coorbital
           chaser.raan == target.raan and
           chaser.a == target.a and
           chaser.w == target.w and 
           chaser.e == target.e)
    r_apogee = mag_of(chaser.get_position_at_time(0)) # chaser is at perigee, so this is true
    
    # we have two basic options
    # option 1, chaser increases period so target can catch up (i.e. stalls for some time)
    total_stall_time = target.get_t_until_v(chaser.v)
    stall_time_per_period = total_stall_time / periods
    stall_period = chaser.period + stall_time_per_period 
    
    # option 2, chaser decreases period so it can catch target (i.e. catches up some time)
    catchup_period = target.get_t_until_v(chaser.v) # we want to be in same place when target arrives
    total_catchup_time = chaser.period - catchup_period
    catchup_time_per_period = total_catchup_time / periods
    catchup_period = chaser.period - catchup_time_per_period # re-adjust so change happens over 
    # multiple periods
    
    phasing_orbit = chaser.duplicate()
    
    # pick whichever takes less effort
    if (stall_time_per_period < catchup_time_per_period):
        phasing_orbit.a = period_to_axis(stall_period)
        phasing_orbit.e = r_apogee_and_a_to_e(r_apogee, phasing_orbit.a) # keep the same apogee
    else:
        phasing_orbit.a = period_to_axis(catchup_period)
        phasing_orbit.e = r_apogee_and_a_to_e(r_apogee, phasing_orbit.a) # keep the same apogee
    phasing_orbit.recomp_meta_vars()
    delta_v = abs(phasing_orbit.get_vel() - chaser.get_vel())
    
    return delta_v * 2 # because we also have to go back to mission orbit

time_elapsed = tof_hohman + time_to_perigee + h_to_sec(48) # the current elapsed time
const3.simulate(time_elapsed) # figure out where the target positions are now
target.v = 180 # we are currently at apogee
tandem_orbit = target # we are starting from the combined orbit now
tandem_orbit.recomp_meta_vars()

# we will end up splitting the phase change maneuver over 48 hours to reduce the requisite delta v
# this can only occur in increments of whole periods, hence the floor
max_allowable_periods = math.floor(h_to_sec(48) / tandem_orbit.period)
required_phasing_time = max_allowable_periods * tandem_orbit.period

total_delta_v_phasing = 0 # start a tracker to add these up
max_delta_v_phasing = 0 # start a tracker to find the max, as well
sat_number = 0
for sat in const3.sats:
    sat_number += 1
    target_pos : MSIII_Orbit = sat.orbit
    this_sat_delta_v = get_total_coorbital_rendezvous_delta_v(tandem_orbit, 
                                                              target_pos, 
                                                              max_allowable_periods)
    print(f"Satellite {sat_number} requires delta V of {this_sat_delta_v} km s^-1")
    total_delta_v_phasing += this_sat_delta_v
    max_delta_v_phasing = max(max_delta_v_phasing, this_sat_delta_v)

print(f"In total, that is {total_delta_v_phasing} km s^-1 of delta v to achieve proper phasing")
print(f"This will occur over the span of {sec_to_h(required_phasing_time)} hours to conserve fuel.")


    

Satellite 1 requires delta V of 0.07371194093663291 km s^-1
Satellite 2 requires delta V of 0.00545072199708585 km s^-1
Satellite 3 requires delta V of 0.01108222888474053 km s^-1
In total, that is 0.0902448918184593 km s^-1 of delta v to achieve proper phasing
This will occur over the span of 47.999999999999964 hours to conserve fuel.


### Step 6

### a.

In [122]:
perigee_test = tandem_orbit.duplicate()
perigee_test.v = 0
perigee_test.recomp_meta_vars()
min_alt = perigee_test.get_R() - R_EARTH
print(f"Perigee altitude: {min_alt} km")


if (math.floor(min_alt / 50) * 50 == 600): # from the table
    annual_drag_delta_v = m_to_km(0.891)
    total_drag_delta_v = MIN_LIFETIME * annual_drag_delta_v
    print(f"Annual delta v to compensate drag: {annual_drag_delta_v} km s^-1")
    print(f"This adds up to {total_drag_delta_v} km s^-1 over {MIN_LIFETIME} years")
else:
    print("FIXME") # we hardcoded in just one value from the table. If that was the wrong value,
    # this notifies the programmer to hardcode a different one

Perigee altitude: 601.0 km
Annual delta v to compensate drag: 0.000891 km s^-1
This adds up to 0.004455 km s^-1 over 5 years


#### b.

In [123]:
if (math.floor(min_alt / 50) * 50 == 600): # from the table
    annual_j2_delta_v = m_to_km(103.22 + 80.50)
    total_j2_delta_v = MIN_LIFETIME * annual_j2_delta_v
    print(f"Annual delta v to compensate J2: {annual_j2_delta_v} km s^-1")
    print(f"This adds up to {total_j2_delta_v} km s^-1 over {MIN_LIFETIME} years")
else:
    print("FIXME") # we hardcoded in just one value from the table. If that was the wrong value,
    # this notifies the programmer to hardcode a different one

Annual delta v to compensate J2: 0.18372 km s^-1
This adds up to 0.9186 km s^-1 over 5 years


#### c. 

In [124]:
# add the two together and print out

lifetime_orbit_maintenance_delta_v = total_j2_delta_v + total_drag_delta_v
print(f"Over the {MIN_LIFETIME} yr lifetime, orbit maintenance requires " + 
      f"{lifetime_orbit_maintenance_delta_v} km s^-1 in delta v")

Over the 5 yr lifetime, orbit maintenance requires 0.923055 km s^-1 in delta v


#### d.

In [125]:

if (math.floor(min_alt / 50) * 50 == 600 and const3.adcs == ADCS[1]): # from the table
    annual_adcs_delta_v = m_to_km(0.669)
    total_adcs_delta_v = MIN_LIFETIME * annual_adcs_delta_v
    print(f"Annual delta v to actuate ADCS: {annual_adcs_delta_v} km s^-1")
    print(f"This adds up to {total_adcs_delta_v} km s^-1 over {MIN_LIFETIME} years")
else:
    print("FIXME") # we hardcoded in just one value from the table. If that was the wrong value,
    # this notifies the programmer to hardcode a different one

Annual delta v to actuate ADCS: 0.000669 km s^-1
This adds up to 0.003345 km s^-1 over 5 years


#### e.

In [126]:
delta_v_asat = m_to_km(300)

### Step 7

#### a.

In [127]:
total_onboard_delta_v = sum([
    standard_delta_v_1,
    standard_delta_v_2,
    max_delta_v_phasing,
    lifetime_orbit_maintenance_delta_v,
    total_adcs_delta_v
])

print(f"Each spacecraft must have {total_onboard_delta_v} km s^-1 of delta v")

Each spacecraft must have 1.4373745160269775 km s^-1 of delta v


#### b.

In [128]:
sat_dry_mass = sum([ # recompute dry mass
    const3.adcs["mass"],
    const3.payload["mass"],
    const3.structure["mass"],
    PROPULSION_MASS, # mass of our chosen propulsion system
    36.75, # mass of all our tcs surfaces
    50 # mass of EPS Option II
])

print(f"Sat Dry Mass: {sat_dry_mass} kg")

mass_ratio = math.exp(total_onboard_delta_v / (PROPULSION_ISP * m_to_km(G_0)))
min_wet_mass = mass_ratio * sat_dry_mass
propellant_mass = min_wet_mass - sat_dry_mass

print(f"We will need at least {propellant_mass} kg of propellant per satellite.")

Sat Dry Mass: 299.75 kg
We will need at least 275.1262652472635 kg of propellant per satellite.


#### c.

In [129]:
# we will keep as little fuel as possible onboard, so we will round up and call it good.
# this will save on cost

onboard_propellant = math.ceil(propellant_mass)
print(f"We will keep {onboard_propellant} kg of fuel onboard each satellite.")

We will keep 276 kg of fuel onboard each satellite.


### Step 8 

In [130]:
wet_mass = onboard_propellant + sat_dry_mass
print(f"The final wet mass of each satellite will be {wet_mass} kg")

The final wet mass of each satellite will be 575.75 kg
